In [44]:
from neo4j import GraphDatabase
import pandas as pd

In [45]:
uri ='bolt://localhost:7687'
user = 'neo4j'
password = '09150915'

driver = GraphDatabase.driver(uri, auth=(user,password))

In [47]:
df_coauthor_LC = pd.read_csv('coauthor_lung_cancer.csv')
df_coauthor_SC = pd.read_csv('coauthor_super_conducter.csv')

In [76]:
create_coauthor_graph(df_coauthor_LC, 'lung_cancer')

In [77]:
create_coauthor_graph(df_coauthor_SC, 'super_conducter')

In [70]:
result = calculate_graph_stats_for_field('super_conducter')

print(result)

{'field': 'super_conducter', 'num_authors': 224, 'num_authors_FastRP': 224, 'num_edges': 206, 'degree_centrality_max': 17.00011444091797, 'degree_centrality_mean': 0.9196454456874302, 'num_community': 166, 'num_authors_in_main_component': 20}


In [ ]:
concat_df()